## MTH9878 Interest Rate Models HW1
* Group X
* Author: Pan, Hongchao & Zhang, Chendi
* Kernel version: Python 2.7
* Packages: 
* Data: 
* Notes:

In [1]:
# load packages and features from version 3.x
from __future__ import absolute_import, print_function, division
import pandas as pd
import numpy as np
import math

### Answer of Q1

In [2]:
# Define a function to calculate the faction of the date differnence
def date_fraction(t1,t2):
    # Assume input t1 and t2 are calendar dates
    days=(t2-t1).days
    # Use the date convention: actual/360
    return days/360  

In [3]:
# Define a funtion to compute the integration by using simpson's rule
# a: left endpoint; b: right endpoint; n: number of partition intervals
# f: instantaneous rate function, f(s)
def simpson_rule(a,b,f,n):
    h= (b-a)/n
    result=(f(a) + f(b))/6

    for i in range(1, n):
        result += f(a + i*h)/3
    for i in range(1, n+1):
        result += 2*f(a + (i-0.5)*h)/3

    return result*h

In [4]:
def approx_val_tol(f_method,a,b,f,n,tol):
    # f_method: integration method: mid_point/trape/simpson method; simpson's rule here
    # tol=5e-7
    # n: number of partition intervals
    # f: instantaneous rate function, f(s)
    # a: left endpoint, t here; b: right endpoint, T here;
    result_old=f_method(a,b,f,n)
    n=2*n
    result_new=f_method(a,b,f,n)

    while(abs(result_old-result_new)>tol):
        result_old=result_new
        n=2*n
        result_new=f_method(a,b,f,n)

    return result_new


In [5]:
# Define the interest rate function
# the constant interest rate
def f_const(x):
    return 0.55/100   

# instantaneous rate, not the case in this homework
def f_instant(x):
    return math.exp(-x^2)

In [6]:
# Define a function to compute the discount factor
def dist(t,T,f,flag):
    # t: starting date
    # T: maturity
    # f: interest rate
    # flag: indicator of f is constant rate or instantaneous rate
    
    # Assume input t and T are calendar dates
    
    # Get the day fraction
    day_frac=date_fraction(t,T)
    if(flag=='c'):
        # constant interest rate
        return math.exp(-f*day_frac)
    if(flag=='i'):
        # instantaneous rate
        # starting intervals: 4
        # tolerance: 5e-7
        # Since t and T are calendar date, convert them to get day fraction period,
        # which is equivalent to 0 to day_frac
        return math.exp(-approx_val_tol(simpson_rule,0,day_frac,f,4,5e-7))
    else:
        print("The inputs of are not well defined.")
        return -2

### Answer of Q2

In [9]:
# define a function to calculate the LIBOR forward rate
# reference formula: equation (12) in lecture 1
def LFR(S,T,I,flag):
    # S: settlement date
    # T: maturity
    # I: instantaneous forward rate
    # flag: indicator of I is constant or function
    
    # get the day fraction
    day_frac=date_fraction(t,T)
   
    if(flag=='c'):
        # constant interest rate
        return (math.exp(I*day_frac)-1)/day_frac  # equation (12) in lecture 1
    if(flag=='i'):
        # instantaneous rate
        # starting intervals: 4
        # tolerance: 5e-7
        # Since t and T are calendar date, convert them to get day fraction period,
        # which is equivalent to 0 to day_frac
        return (math.exp(approx_val_tol(simpson_rule,0,day_frac,I,4,5e-7))-1)/day_frac
    else:
        print("The inputs are not well defined.")
        return -2
    